In [20]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import os

## Chargement des Données

In [5]:
# Les données sont accessibles dans "/kaggle/input/".
# Ici, on affiche le contenu du dossier et des sous-dossiers
data_dir = './data/'

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [24]:
# Lecture des données d'entraînement
train_data = pd.read_csv(os.path.join(data_dir, 'snli_train.csv'))
train_data.head()

train_data, validation_data = train_test_split(train_data, test_size=0.2)
train_data.head()

,sentence1,sentence2,label1,id
100763,A person riding a dirt bike on mud,A person is on their dirt bike.,entailment,100766
255342,A muzzled greyhound dog wearing yellow and bla...,Three horses compete in a race.,contradiction,255345
193862,A group of people look on as a large bunch of ...,A bunch of people are about to get in the water.,entailment,193865
329443,Girls with orange hair standing on the street ...,A girl with orange hair riding a bicycle.,contradiction,329449
96995,Two middle-aged men try to operate what looks ...,Two guys over 30 are fiddling with a magnifier.,entailment,96998


## Prétraitement

## Plongement

## Entrainement du modèle

In [ ]:
# Ici, vous devez écrire votre modèle

# Le format idéalisé ressemblerait à :
#
# model = NLPModel
# model.learn(train_data)
# submission['score'] = model.predict(test_data)

## Validation du modèle

In [ ]:
test_data = pd.read_csv(os.path.join(data_dir, 'snli_test.csv'))
test_data.head()

# Dans ce notebook, on fait juste des prédictions aléatoires 
labels = train_data.label1.unique()

test_data["label1"] = np.random.choice(labels, size=len(test_data))
submission = test_data[["id", "label1"]]
submission.head()

In [ ]:
# calcul de l'accuracy

## Enregistrement de la soumission

In [10]:
# On sauvegarde nos prédictions dans un fichier
# On peut alors soumettre ce fichier à Kaggle pour l'évaluation
submission.to_csv('./out/submission.csv', index=False)